In [1]:
# Dependencies
import json
import requests
import pandas as pd
import plotly.offline as off
import plotly.plotly as py
off.init_notebook_mode(connected=True)
from pytrends.request import TrendReq

# Google Developer API Key
from config import gkey

In [2]:
# Census Data Collected from csv's
income = pd.read_csv("Resources/income.csv")
age = pd.read_csv("Resources/age.csv")

In [3]:
# Setup Dataframe to Collect Data from CSVs
stats_df = pd.DataFrame()

stats_df["City-State"] = age["Geography"]
stats_df["Total Population"] = age["Total; Estimate; Total population"]
stats_df["Target Population"] = age["Total; Estimate; AGE - 20 to 24 years"] + age["Total; Estimate; AGE - 25 to 29 years"] + age["Total; Estimate; AGE - 30 to 34 years"] + age["Total; Estimate; AGE - 30 to 34 years"] + age["Total; Estimate; AGE - 35 to 39 years"]
stats_df["Median Income"] = income["Median income (dollars); Estimate; HOUSEHOLD INCOME BY AGE OF HOUSEHOLDER - 25 to 44 years"].map(
    "${:,.2f}".format)
#stats_df.head()

In [4]:
# Build Keywords from Suggested Search
keywords=[]
pytrend = TrendReq(hl='en-US')
suggestions = pytrend.suggestions(keyword = 'Brewery')
for item in range(0,len(suggestions)):
    keywords.append(suggestions[item]['title'])

In [5]:
# List for Data Collection
lat = []
long = []
results = []

# Create Dataframe for API Calls
city_df = stats_df
city_df["City-URL-String"] = city_df["City-State"].str.replace("Metro Area","")
city_df["City-URL-String"] = city_df["City-URL-String"].str.replace(" ","+")
#city_df

In [6]:
# Collects the API Results for Each City
for row in range(0,len(city_df)):
    city = city_df["City-URL-String"].loc[row]
    
    # Prints what current city is being called by the API
    print(city)
    
    # Create String for API
    url_string_call =  f"https://maps.googleapis.com/maps/api/geocode/json?address={city}&key={gkey}"
    brewery_data = requests.get(url_string_call)
    
    # Formats Latitude and Longitude Data for Second API Call
    test_var = brewery_data.json()
    target_city = test_var["results"][0]['geometry']['location']
    target_coords = f"{target_city['lat']},{target_city['lng']}"
    params = {
        "location": target_coords,
        "radius": 8000,
        "keyword": keywords,
        "key": gkey
    }
    
    # Second API Call
    base_url = "https://maps.googleapis.com/maps/api/place/radarsearch/json"
    brewery_data = requests.get(base_url, params=params)
    brewery_data = brewery_data.json()

    # Saves Results Data into list to be converted into Dataframes 
    results.append(len(brewery_data["results"]))
    print(len(brewery_data["results"]))
    lat.append(target_city['lat'])
    long.append(target_city['lng'])

Abilene,+TX+
186
Akron,+OH+
132
Albany,+GA+
147
Albany,+OR+
48
Albany-Schenectady-Troy,+NY+
110
Albuquerque,+NM+
163
Alexandria,+LA+
174
Allentown-Bethlehem-Easton,+PA-NJ+
167
Altoona,+PA+
139
Amarillo,+TX+
158
Ames,+IA+
186
Anchorage,+AK+
161
Ann+Arbor,+MI+
152
Anniston-Oxford-Jacksonville,+AL+
70
Appleton,+WI+
195
Asheville,+NC+
193
Athens-Clarke+County,+GA+
122
Atlanta-Sandy+Springs-Roswell,+GA+
107
Atlantic+City-Hammonton,+NJ+
61
Auburn-Opelika,+AL+
91
Augusta-Richmond+County,+GA-SC+
53
Austin-Round+Rock,+TX+
47
Bakersfield,+CA+
183
Baltimore-Columbia-Towson,+MD+
127
Bangor,+ME+
200
Barnstable+Town,+MA+
166
Baton+Rouge,+LA+
188
Battle+Creek,+MI+
197
Bay+City,+MI+
191
Beaumont-Port+Arthur,+TX+
176
Beckley,+WV+
143
Bellingham,+WA+
138
Bend-Redmond,+OR+
16
Billings,+MT+
193
Binghamton,+NY+
186
Birmingham-Hoover,+AL+
108
Bismarck,+ND+
177
Blacksburg-Christiansburg-Radford,+VA+
132
Bloomington,+IL+
117
Bloomington,+IN+
197
Bloomsburg-Berwick,+PA+
57
Boise+City,+ID+
182
Boston-Cambridge-

190
Springfield,+OH+
156
State+College,+PA+
124
Staunton-Waynesboro,+VA+
59
Stockton-Lodi,+CA+
125
Sumter,+SC+
148
Syracuse,+NY+
191
Tallahassee,+FL+
187
Tampa-St.+Petersburg-Clearwater,+FL+
97
Terre+Haute,+IN+
180
Texarkana,+TX-AR+
197
The+Villages,+FL+
97
Toledo,+OH+
116
Topeka,+KS+
198
Trenton,+NJ+
84
Tucson,+AZ+
185
Tulsa,+OK+
116
Tuscaloosa,+AL+
198
Tyler,+TX+
193
Urban+Honolulu,+HI+
197
Utica-Rome,+NY+
186
Valdosta,+GA+
171
Vallejo-Fairfield,+CA+
63
Victoria,+TX+
128
Vineland-Bridgeton,+NJ+
40
Virginia+Beach-Norfolk-Newport+News,+VA-NC+
145
Visalia-Porterville,+CA+
11
Waco,+TX+
194
Walla+Walla,+WA+
190
Warner+Robins,+GA+
185
Washington-Arlington-Alexandria,+DC-VA-MD-WV+
134
Waterloo-Cedar+Falls,+IA+
112
Watertown-Fort+Drum,+NY+
168
Wausau,+WI+
196
Weirton-Steubenville,+WV-OH+
147
Wenatchee,+WA+
184
Wheeling,+WV-OH+
158
Wichita,+KS+
175
Wichita+Falls,+TX+
184
Williamsport,+PA+
161
Wilmington,+NC+
135
Winchester,+VA-WV+
152
Winston-Salem,+NC+
195
Worcester,+MA-CT+
112
Yakima,+WA+
1

In [7]:
# Add Collected Data to DataFrame
city_df['results']=pd.DataFrame({'results':results})
city_df['lat']=pd.DataFrame({'lat':lat})
city_df['long']=pd.DataFrame({'long':long})

# Create Ratio Column to Weigh Results from Population and Google Results
city_df['Ratio']=city_df['Target Population']/(city_df['results'])

# Remove Data that Has Errors
city_clean_df = city_df[city_df['results'] != 0]
df = city_clean_df
city_clean_df.sort_values(['results'])
city_clean_df

,City-State,Total Population,Target Population,Median Income,City-URL-String,results,lat,long,Ratio
0,"Abilene, TX Metro Area",168774,36.9,"$53,963.00","Abilene,+TX+",186,32.448736,-99.733144,0.198387
1,"Akron, OH Metro Area",703168,31.4,"$57,276.00","Akron,+OH+",132,41.081445,-81.519005,0.237879
2,"Albany, GA Metro Area",154830,32.6,"$37,589.00","Albany,+GA+",147,31.578507,-84.155741,0.221769
3,"Albany, OR Metro Area",119862,31.4,"$53,612.00","Albany,+OR+",48,44.636511,-123.105928,0.654167
4,"Albany-Schenectady-Troy, NY Metro Area",879291,32.5,"$68,171.00","Albany-Schenectady-Troy,+NY+",110,42.763648,-73.653621,0.295455
5,"Albuquerque, NM Metro Area",904486,34.3,"$50,756.00","Albuquerque,+NM+",163,35.084386,-106.650422,0.210429
6,"Alexandria, LA Metro Area",154745,32.8,"$44,119.00","Alexandria,+LA+",174,31.311294,-92.445137,0.188506
7,"Allentown-Bethlehem-Easton, PA-NJ Metro Area",830737,29.9,"$66,996.00","Allentown-Bethlehem-Easton,+PA-NJ+",167,40.729185,-75.247906,0.179042
8,"Altoona, PA Metro Area",125917,29.1,"$54,733.00","Altoona,+PA+",139,40.518681,-78.394736,0.209353
9,"Amarillo, TX Metro Area",260495,35.6,"$54,958.00","Amarillo,+TX+",158,35.221997,-101.831297,0.225316


In [8]:
df['text'] = city_clean_df['City-State'] + '<br>Population ' + (city_clean_df['Total Population']/1e6).astype(str)+' million'

colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
cities = []
scale = 10

/home/chuck/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [9]:

df_sub = city_clean_df
df_sub['scale']=((df_sub['Target Population']/scale)*25)
city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['scale'],
        color = colors[0],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'City Population')
cities.append(city)


/home/chuck/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [10]:

df_sub = city_clean_df

city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['results'],
        color = colors[1],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'Google Map Results')
cities.append(city)


In [11]:

df_sub = city_clean_df

city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['Ratio']*100,
        color = colors[2],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'Ratio of Population by Results')
cities.append(city)

layout = dict(
        title = 'USA Brewery Data<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
off.iplot( fig, validate=False, filename='d3-bubble-map-populations' )

In [12]:
top_five_df = city_clean_df.sort_values(['Ratio'], ascending=False).head()
top_five_df

,City-State,Total Population,Target Population,Median Income,City-URL-String,results,lat,long,Ratio,text,scale
239,"Mount Vernon-Anacortes, WA Metro Area",120475,29.9,"$59,405.00","Mount+Vernon-Anacortes,+WA+",1,48.424196,-121.711436,29.900000,"Mount Vernon-Anacortes, WA Metro Area<br>Popul...",74.75
325,"Sierra Vista-Douglas, AZ Metro Area",128177,31.0,"$53,383.00","Sierra+Vista-Douglas,+AZ+",2,31.828458,-109.949686,15.500000,"Sierra Vista-Douglas, AZ Metro Area<br>Populat...",77.50
148,"Hanford-Corcoran, CA Metro Area",150261,41.0,"$47,954.00","Hanford-Corcoran,+CA+",3,36.098849,-119.881520,13.666667,"Hanford-Corcoran, CA Metro Area<br>Population ...",102.50
191,"Lake Havasu City-Kingman, AZ Metro Area",203629,24.0,"$46,115.00","Lake+Havasu+City-Kingman,+AZ+",2,35.214335,-113.763283,12.000000,"Lake Havasu City-Kingman, AZ Metro Area<br>Pop...",60.00
149,"Harrisburg-Carlisle, PA Metro Area",561447,31.7,"$63,441.00","Harrisburg-Carlisle,+PA+",9,40.315705,-77.102490,3.522222,"Harrisburg-Carlisle, PA Metro Area<br>Populati...",79.25


In [13]:
df['text'] = city_clean_df['City-State'] + '<br>Population ' + (city_clean_df['Total Population']/1e6).astype(str)+' million'

cities = []
df_sub = top_five_df

city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['scale'],
        color = colors[0],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'Top 5 City Population')
cities.append(city)

city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['results'],
        color = colors[1],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'Google Map Results')
cities.append(city)

city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['Ratio']*100,
        color = colors[2],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'Ratio of Population by Results')
cities.append(city)

/home/chuck/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [14]:
layout = dict(
        title = 'USA Brewery Data<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
off.iplot( fig, validate=False, filename='d3-bubble-map-populations' )